In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, f1_score, precision_score, recall_score, roc_auc_score,
    confusion_matrix, classification_report
)

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
from keras import Input, Model
from keras.layers import (
    Embedding, Bidirectional, LSTM, Dropout, BatchNormalization,
    Conv1D, MaxPooling1D, GlobalMaxPooling1D, Flatten, Dense, GRU
)
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import gc
#warnings
import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
splits = {'train': 'cleaned_train_multi.csv', 'test': 'cleaned_test_multi.csv'}
train_df = pd.read_csv("hf://datasets/kcrl/Shared_Task_Fake_News_Multi/" + splits["train"])
test_df = pd.read_csv("hf://datasets/kcrl/Shared_Task_Fake_News_Multi/" + splits["test"])

In [ ]:
with pd.option_context('display.max_colwidth',0):
    display(train_df.sample(n=10))

,ID,News,Label
701,FAKE_MAL_TR_0702,കെഎസ്ആര്‍ടിസി ഫീഡര്‍ ബസ് കണ്ട് ഭയന്ന് ഓടി വീണ് മൂന്ന് പേര്‍ക്ക് പരുക്കേറ്റു,FALSE
61,FAKE_MAL_TR_0062,യൂണിഫോമിനൊപ്പം താടി നീട്ടി വളര്‍ത്തിയ കേരള പൊലീസ് ഉദ്യോഗസ്ഥന്‍.,FALSE
1061,FAKE_MAL_TR_1062,പഴയ എഞ്ചിനുമായി വന്ദേ ഭാരത് എക്‌സ്പ്രസ്സ്,FALSE
162,FAKE_MAL_TR_0163,കൊല്ലത്ത് ലെയ്‌സ് ചോദിച്ചിട്ട് നല്‍കിയില്ലെന്ന കാരണത്താല്‍ 19കാരനെ മര്‍ദ്ദിച്ച സംഘം സിപിഎം പ്രവര്‍ത്തകരാണ്,FALSE
163,FAKE_MAL_TR_0164,തിരഞ്ഞെടുപ്പുസമയത്ത് ഉമ്മന്‍ചാണ്ടിക്കു വേണ്ടി പുതുപ്പള്ളിയിലും ബിന്ദു കൃഷ്ണയ്ക്കു വേണ്ടി കൊല്ലത്തും സംസാരിച്ച വനിതാ പ്രവര്‍ത്തകത്തന്നെയാണ് കെ-റെയില്‍ വിരുദ്ധ സമരത്തിനെത്തിയത്.,FALSE
539,FAKE_MAL_TR_0540,അതിർത്തിയിൽ ഇന്ത്യൻ സൈനികരോട് കയർക്കുന്ന ചൈനീസ് പട്ടാളം; വീഡിയോയ്ക്ക് പിന്നിലെ സത്യമിത്,MOSTLY FALSE
1249,FAKE_MAL_TR_1250,കോവിഡ് -19 എംആർഎൻഎ വാക്സിനുകളിൽ നിന്നുള്ള സ്പൈക്ക് പ്രോട്ടീൻ അപകടകരമാണ്,FALSE
145,FAKE_MAL_TR_0146,"തള്ളിയിട്ട് 4 വര്‍ഷം, ഇപ്പോഴും നിപ വൈറസ് പരിശോധിക്കാന്‍ പൂനെയില്‍ ക്യൂ നില്‍ക്കണം'; സംഭവം എത്രത്തോളം ശരി?",FALSE
1019,FAKE_MAL_TR_1020,claim by പീഡിപ്പിക്കപ്പെട്ട 5 വയസ്സുകാരിയേയും കൊണ്ട് പാർലിമെന്റിന് ചുറ്റും അലറി വിളിച്ചുകൊണ്ട് നടക്കേണ്ടി വരുന്ന ഒരു പിതാവിന്റെ അവസ്ഥ. ഈ രാജ്യത്ത് സാധാരണക്കാരന്റെ ജീവിതം എങ്ങോട്ടാണെന്നതിന്റെ വ്യക്തമായ സൂചനയാണ്. ബേട്ടീ ബച്ചാവോ :,FALSE
1001,FAKE_MAL_TR_1002,ഇ ശ്രീധരനെ ഐക്യരാഷ്ട്രസഭ ഉപദേശക സമിതി അംഗമായി നിയമിച്ചു,HALF TRUE


In [ ]:
train_df.Label.unique()

array(['FALSE', 'HALF TRUE', 'MOSTLY FALSE', 'PARTLY FALSE'], dtype=object)

In [ ]:
labeling = {
    'FALSE': 0,
    'HALF TRUE': 1,
    'MOSTLY FALSE': 2,
    'PARTLY FALSE': 3,
    'False': 0
}
# Convert 'Label' column to string type to ensure consistency
train_df['Label'] = train_df['Label'].astype(str)
test_df['Label'] = test_df['Label'].astype(str)
# Apply the mapping only to non-numeric labels
train_df['Label'] = train_df['Label'].apply(lambda x: labeling[x] if x in labeling else x)
test_df['Label'] = test_df['Label'].apply(lambda x: labeling[x] if x in labeling else x)

In [ ]:
with pd.option_context('display.max_colwidth',0):
    display(train_df.sample(n=5))

,ID,News,Label
1268,FAKE_MAL_TR_1269,"സ്ത്രീ സുരക്ഷ അടിയന്തര പ്രമേയത്തിന് നോട്ടീസ് നല്‍കിയത് എൽദോസ് കുന്നപ്പിള്ളി പി കെ കുഞ്ഞാലികുട്ടി, എം വിൻസെന്റ് എന്നീ എംഎൽഎമാരോ? വസ്തുത അറിയുക",0
727,FAKE_MAL_TR_0728,പോപ്പുലര്‍ ഫ്രണ്ട് ഹര്‍ത്താല്‍ പിന്‍വലിച്ചു,0
292,FAKE_MAL_TR_0293,ഏഴാം അറിവിൽ ജൂനിയർ ആർട്ടിസ്റ്റായി അപർണ്ണ ബാലമുരളി അഭിനയിച്ചിട്ടുണ്ട്.,0
542,FAKE_MAL_TR_0543,വെന്‍റിലേറ്ററിലേക്ക് മാറ്റും മുൻപുള്ള ഡോ.അയിഷയുടെ വാക്കുകൾ; വൈറലായ സന്ദേശം വ്യാജം,0
1462,FAKE_MAL_TR_1463,കുരുക്ഷേത്രയിൽ ആരതി നടത്തുന്നതിനിടെ ഫോട്ടോഗ്രാഫർമാർക്കായി രാഹുൽ ഗാന്ധി നിർത്തി,0


In [ ]:
from sklearn.utils import resample

# Set the target number of instances for minority classes
target_minority_count = 200

for minority_class in [1,2,3]:

  # Check if the minority class exists in the DataFrame
  if train_df['Label'].isin([minority_class]).any():
    minority_indices = train_df[train_df['Label'] == minority_class].index
    minority_data = train_df.loc[minority_indices]

    minority_oversampled = resample(
        minority_data,
        replace=True,
        n_samples=target_minority_count,
        random_state=42
    )

    train_df = pd.concat([train_df, minority_oversampled], ignore_index=True)

In [ ]:

X = train_df['News']
y = train_df['Label']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:

X_test = test_df['News']
y_test = test_df['Label']

In [ ]:
# Tokenizing the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
vocab=tokenizer.word_index

In [ ]:
train_sequence = tokenizer.texts_to_sequences(X_train)
val_sequence = tokenizer.texts_to_sequences(X_val)
test_sequence = tokenizer.texts_to_sequences(X_test)

In [ ]:
maxlen = 120
train_data = pad_sequences(train_sequence, maxlen=maxlen,  padding='post')
val_data = pad_sequences(val_sequence, maxlen=maxlen,  padding='post')
test_data = pad_sequences(test_sequence, maxlen=maxlen,  padding='post')

In [ ]:
train_labels = y_train
val_labels = y_val
test_labels = y_test

In [ ]:
train_labels_reshaped = np.expand_dims(train_labels, axis=-1)
val_labels_reshaped = np.expand_dims(val_labels, axis=-1)
test_labels_reshaped = np.expand_dims(test_labels, axis=-1)

In [ ]:
vocab_size = len(tokenizer.word_index) + 1
input_shape = (maxlen,)

In [ ]:
from keras.models import Model
from keras.layers import Input, Embedding, LSTM, Dropout, BatchNormalization, Dense, Flatten
from keras.regularizers import l2

# Define the embedding layer
embedding_layer = Embedding(
    input_dim=vocab_size,
    output_dim=300,
    input_length=maxlen,
    trainable=True  # Allow fine-tuning of embeddings
)

# Define the input layer
email_input = Input(shape=(maxlen,), name='Email_input')

# Apply the embedding layer to the input
embedded_email = embedding_layer(email_input)

# Define the Bidirectional LSTM layers
lstm_layer1 = LSTM(units=64, return_sequences=True)
lstm_layer2 = LSTM(units=32, return_sequences=True)

# Apply the first LSTM layer
lstm_output1 = lstm_layer1(embedded_email)

# Apply dropout after the first LSTM layer
dropout1 = Dropout(0.25)(lstm_output1)

# Apply batch normalization
bn1 = BatchNormalization()(dropout1)

# Apply the second LSTM layer
lstm_output2 = lstm_layer2(bn1)

# Flatten the output
flatten_output = Flatten()(lstm_output2)

# Add a dense layer for multiclass classification
output = Dense(4, activation='softmax', kernel_regularizer=l2(0.01))(flatten_output)

# Create the model
lstm_model = Model(inputs=email_input, outputs=output)

# Compile the model
lstm_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Display the model summary
lstm_model.summary()


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ Email_input (InputLayer)             │ (None, 120)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_3 (Embedding)              │ (None, 120, 300)            │       2,607,300 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_6 (LSTM)                        │ (None, 120, 64)             │          93,440 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 120, 64)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 120, 64)             │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_7 (LSTM)                        │ (None, 120, 32)             │          12,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_3 (Flatten)                  │ (None, 3840)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 4)                   │          15,364 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,728,776 (10.41 MB)

 Trainable params: 2,728,648 (10.41 MB)

 Non-trainable params: 128 (512.00 B)

In [ ]:
stopper= EarlyStopping(patience=5, monitor='val_accuracy')
checkpointer = ModelCheckpoint(filepath='model.weights.best.keras', verbose=3, save_best_only=True)

history=lstm_model.fit(
         train_data, train_labels_reshaped,
         batch_size=64,
         epochs=15,
         validation_data=(val_data, val_labels_reshaped),
         callbacks=[stopper, checkpointer],
         )

Epoch 1/15
30/32 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.5470 - loss: 1.2300
Epoch 1: val_loss improved from inf to 1.34869, saving model to model.weights.best.keras
32/32 ━━━━━━━━━━━━━━━━━━━━ 4s 39ms/step - accuracy: 0.5505 - loss: 1.2213 - val_accuracy: 0.5600 - val_loss: 1.3487
Epoch 2/15
31/32 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.8924 - loss: 0.4433
Epoch 2: val_loss improved from 1.34869 to 1.18215, saving model to model.weights.best.keras
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step - accuracy: 0.8936 - loss: 0.4384 - val_accuracy: 0.8220 - val_loss: 1.1821
Epoch 3/15
31/32 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.9927 - loss: 0.1150
Epoch 3: val_loss improved from 1.18215 to 1.06992, saving model to model.weights.best.keras
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 0.9926 - loss: 0.1145 - val_accuracy: 0.8520 - val_loss: 1.0699
Epoch 4/15
31/32 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.9978 - loss: 0.0660
Epoch 4: val_loss improved from 1.069

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Get predictions (class indices for each sample)
pred = lstm_model.predict(test_data)
y_pred = np.argmax(pred, axis=1)  # Get the class with the highest probability

# Calculate metrics
precision = precision_score(y_test, y_pred, average='micro')  # Micro-average precision
recall = recall_score(y_test, y_pred, average='micro')        # Micro-average recall
f1 = f1_score(y_test, y_pred, average='micro')                # Micro-average F1 score
accuracy = accuracy_score(y_test, y_pred)                     # Accuracy

# Print results
print("Precision (Micro):", precision)
print("Recall (Micro):", recall)
print("F1 Score (Micro):", f1)
print("Accuracy:", accuracy)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
Precision (Micro): 0.62
Recall (Micro): 0.62
F1 Score (Micro): 0.62
Accuracy: 0.62


In [ ]:
embedding_layer = Embedding(
    input_dim=vocab_size,
    output_dim=300,
    input_length=maxlen,
    trainable=True  # Allow fine-tuning of embeddings
)
# Define the input layer
email_input = Input(shape=(maxlen,), name='Email_input')

# Apply the embedding layer to the input
embedded_email = embedding_layer(email_input)

# Define the Bidirectional LSTM layers
lstm_layer1 = Bidirectional(LSTM(units=256, return_sequences=True))
lstm_layer2 =Bidirectional(LSTM(units=128, return_sequences=True))

# Apply the first LSTM layer
lstm_output1 = lstm_layer1(embedded_email)

# Apply dropout after the first LSTM layer
dropout1 = Dropout(0.25)(lstm_output1)

# Apply batch normalization
bn1 = BatchNormalization()(dropout1)

# Apply the second LSTM layer
lstm_output2 = lstm_layer2(bn1)

flatten_output = Flatten()(lstm_output2)

# Add a dense layer for binary classification
output = Dense(7, activation='softmax', kernel_regularizer=l2(0.001))(flatten_output)

# Create the model
bilstm_model = Model(inputs=email_input, outputs=output)

# Compile the model
bilstm_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Display the model summary
bilstm_model.summary()


Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ Email_input (InputLayer)             │ (None, 120)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_4 (Embedding)              │ (None, 120, 300)            │       2,607,300 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_2 (Bidirectional)      │ (None, 120, 512)            │       1,140,736 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 120, 512)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_4                │ (None, 120, 512)            │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_3 (Bidirectional)      │ (None, 120, 256)            │         656,384 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_4 (Flatten)                  │ (None, 30720)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 7)                   │         215,047 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,621,515 (17.63 MB)

 Trainable params: 4,620,491 (17.63 MB)

 Non-trainable params: 1,024 (4.00 KB)

In [ ]:
stopper= EarlyStopping(patience=5, monitor='val_accuracy')
checkpointer = ModelCheckpoint(filepath='model.weights.best.keras', verbose=3, save_best_only=True)

history=bilstm_model.fit(
         train_data, train_labels_reshaped,
         batch_size=64,
         epochs=15,
         validation_data=(val_data, val_labels_reshaped),
         callbacks=[stopper, checkpointer],
         )

Epoch 1/15
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - accuracy: 0.4409 - loss: 1.4022
Epoch 1: val_loss improved from inf to 1.40786, saving model to model.weights.best.keras
32/32 ━━━━━━━━━━━━━━━━━━━━ 11s 146ms/step - accuracy: 0.4435 - loss: 1.3966 - val_accuracy: 0.5620 - val_loss: 1.4079
Epoch 2/15
31/32 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 0.8610 - loss: 0.3998
Epoch 2: val_loss did not improve from 1.40786
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 62ms/step - accuracy: 0.8638 - loss: 0.3944 - val_accuracy: 0.5780 - val_loss: 1.6661
Epoch 3/15
31/32 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 0.9843 - loss: 0.0755
Epoch 3: val_loss improved from 1.40786 to 1.26374, saving model to model.weights.best.keras
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 72ms/step - accuracy: 0.9842 - loss: 0.0757 - val_accuracy: 0.5540 - val_loss: 1.2637
Epoch 4/15
31/32 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 0.9971 - loss: 0.0327
Epoch 4: val_loss did not improve from 1.26374
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 62ms

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Get predictions (class indices for each sample)
pred = bilstm_model.predict(test_data)
y_pred = np.argmax(pred, axis=1)  # Get the class with the highest probability

# Calculate metrics
precision = precision_score(y_test, y_pred, average='micro')  # Micro-average precision
recall = recall_score(y_test, y_pred, average='micro')        # Micro-average recall
f1 = f1_score(y_test, y_pred, average='micro')                # Micro-average F1 score
accuracy = accuracy_score(y_test, y_pred)                     # Accuracy

# Print results
print("Precision (Micro):", precision)
print("Recall (Micro):", recall)
print("F1 Score (Micro):", f1)
print("Accuracy:", accuracy)


7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 92ms/step
Precision (Micro): 0.505
Recall (Micro): 0.505
F1 Score (Micro): 0.505
Accuracy: 0.505


In [ ]:
embedding_layer = Embedding(
    input_dim=vocab_size,
    output_dim=300,
    input_length=maxlen,
    trainable=True  # Allow fine-tuning of embeddings
)
email_input = Input(shape=(maxlen,), name='Email_input')

# Apply the embedding layer to the input
embedded_email = embedding_layer(email_input)

# Define and apply the Bidirectional LSTM layer
lstm_layer = Bidirectional(LSTM(units=256, return_sequences=True))(embedded_email)

# Apply dropout after LSTM
dropout1 = Dropout(0.25)(lstm_layer)

# Apply batch normalization
bn1 = BatchNormalization()(dropout1)

# Add CNN layers after BiLSTM
# First CNN block
conv1 = Conv1D(filters=64, kernel_size=3, activation='relu', padding='same')(bn1)
pool1 = MaxPooling1D(pool_size=2)(conv1)

# Second CNN block
conv2 = Conv1D(filters=64, kernel_size=3, activation='relu', padding='same')(pool1)
pool2 = MaxPooling1D(pool_size=2)(conv2)

# Apply global max pooling
global_pool = GlobalMaxPooling1D()(pool2)

# Add dropout for regularization
dropout2 = Dropout(0.25)(global_pool)

# Add batch normalization
bn2 = BatchNormalization()(dropout2)

# Add a dense layer for binary classification
output = Dense(7, activation='softmax', kernel_regularizer=l2(0.001))(bn2)

# Create the model
bilstm_cnn_model = Model(inputs=email_input, outputs=output)

# Compile the model
bilstm_cnn_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Display the model summary
bilstm_cnn_model.summary()

Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ Email_input (InputLayer)             │ (None, 120)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_6 (Embedding)              │ (None, 120, 300)            │       2,607,300 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_5 (Bidirectional)      │ (None, 120, 512)            │       1,140,736 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 120, 512)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_7                │ (None, 120, 512)            │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_2 (Conv1D)                    │ (None, 120, 64)             │          98,368 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_2 (MaxPooling1D)       │ (None, 60, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_3 (Conv1D)                    │ (None, 60, 64)              │          12,352 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_3 (MaxPooling1D)       │ (None, 30, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_max_pooling1d_1               │ (None, 64)                  │               0 │
│ (GlobalMaxPooling1D)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_8 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_8                │ (None, 64)                  │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 7)                   │             455 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,861,515 (14.73 MB)

 Trainable params: 3,860,363 (14.73 MB)

 Non-trainable params: 1,152 (4.50 KB)

In [ ]:
stopper= EarlyStopping(patience=5, monitor='val_accuracy')
checkpointer = ModelCheckpoint(filepath='model.weights.best.keras', verbose=3, save_best_only=True)

history=bilstm_cnn_model.fit(
         train_data, train_labels_reshaped,
         batch_size=64,
         epochs=15,
         validation_data=(val_data, val_labels_reshaped),
         callbacks=[stopper, checkpointer],
         )

Epoch 1/15
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.1675 - loss: 2.6136
Epoch 1: val_loss improved from inf to 1.91667, saving model to model.weights.best.keras
32/32 ━━━━━━━━━━━━━━━━━━━━ 12s 78ms/step - accuracy: 0.1676 - loss: 2.6116 - val_accuracy: 0.1800 - val_loss: 1.9167
Epoch 2/15
31/32 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.2169 - loss: 2.3090
Epoch 2: val_loss did not improve from 1.91667
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - accuracy: 0.2178 - loss: 2.3056 - val_accuracy: 0.1800 - val_loss: 2.0258
Epoch 3/15
31/32 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.4057 - loss: 1.6494
Epoch 3: val_loss did not improve from 1.91667
32/32 ━━━━━━━━━━━━━━━━━━━━ 3s 44ms/step - accuracy: 0.4105 - loss: 1.6368 - val_accuracy: 0.1980 - val_loss: 2.0526
Epoch 4/15
31/32 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.6617 - loss: 0.8892
Epoch 4: val_loss did not improve from 1.91667
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - accuracy: 0.6635 - loss: 0.8852 - val_a

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Get predictions (class indices for each sample)
pred = bilstm_cnn_model.predict(test_data)
y_pred = np.argmax(pred, axis=1)  # Get the class with the highest probability

# Calculate metrics
precision = precision_score(y_test, y_pred, average='micro')  # Micro-average precision
recall = recall_score(y_test, y_pred, average='micro')        # Micro-average recall
f1 = f1_score(y_test, y_pred, average='micro')                # Micro-average F1 score
accuracy = accuracy_score(y_test, y_pred)                     # Accuracy

# Print results
print("Precision (Micro):", precision)
print("Recall (Micro):", recall)
print("F1 Score (Micro):", f1)
print("Accuracy:", accuracy)


7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 75ms/step
Precision (Micro): 0.61
Recall (Micro): 0.61
F1 Score (Micro): 0.61
Accuracy: 0.61


In [ ]:
embedding_layer = Embedding(
    input_dim=vocab_size,
    output_dim=300,
    input_length=maxlen,
    trainable=True  # Allow fine-tuning of embeddings
)
email_input = Input(shape=(maxlen,), name='Email_input')

# Apply the embedding layer to the input
embedded_email = embedding_layer(email_input)

# Define and apply the Bidirectional LSTM layer
lstm_layer = Bidirectional(GRU(units=128, return_sequences=True))(embedded_email)

# Apply dropout after LSTM
dropout1 = Dropout(0.25)(lstm_layer)

# Apply batch normalization
bn1 = BatchNormalization()(dropout1)

# Add CNN layers after BiLSTM
# First CNN block
conv1 = Conv1D(filters=128, kernel_size=3, activation='relu', padding='same')(bn1)
pool1 = MaxPooling1D(pool_size=2)(conv1)

# Second CNN block
conv2 = Conv1D(filters=64, kernel_size=3, activation='relu', padding='same')(pool1)
pool2 = MaxPooling1D(pool_size=2)(conv2)

# Apply global max pooling
global_pool = GlobalMaxPooling1D()(pool2)

# Add dropout for regularization
dropout2 = Dropout(0.25)(global_pool)

# Add batch normalization
bn2 = BatchNormalization()(dropout2)

# Add a dense layer for binary classification
output = Dense(7, activation='softmax', kernel_regularizer=l2(0.001))(bn2)

# Create the model
bigru_cnn_model = Model(inputs=email_input, outputs=output)

# Compile the model
bigru_cnn_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Display the model summary
bigru_cnn_model.summary()

Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ Email_input (InputLayer)             │ (None, 120)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_7 (Embedding)              │ (None, 120, 300)            │       2,607,300 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_6 (Bidirectional)      │ (None, 120, 256)            │         330,240 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_9 (Dropout)                  │ (None, 120, 256)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_9                │ (None, 120, 256)            │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_4 (Conv1D)                    │ (None, 120, 128)            │          98,432 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_4 (MaxPooling1D)       │ (None, 60, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_5 (Conv1D)                    │ (None, 60, 64)              │          24,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_5 (MaxPooling1D)       │ (None, 30, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_max_pooling1d_2               │ (None, 64)                  │               0 │
│ (GlobalMaxPooling1D)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_10 (Dropout)                 │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_10               │ (None, 64)                  │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 7)                   │             455 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,062,347 (11.68 MB)

 Trainable params: 3,061,707 (11.68 MB)

 Non-trainable params: 640 (2.50 KB)

In [ ]:
stopper= EarlyStopping(patience=5, monitor='val_accuracy')
checkpointer = ModelCheckpoint(filepath='model.weights.best.keras', verbose=3, save_best_only=True)

history=bigru_cnn_model.fit(
         train_data, train_labels_reshaped,
         batch_size=64,
         epochs=15,
         validation_data=(val_data, val_labels_reshaped),
         callbacks=[stopper, checkpointer],
         )

Epoch 1/15
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.1562 - loss: 2.4865
Epoch 1: val_loss improved from inf to 2.27375, saving model to model.weights.best.keras
32/32 ━━━━━━━━━━━━━━━━━━━━ 11s 119ms/step - accuracy: 0.1573 - loss: 2.4825 - val_accuracy: 0.1020 - val_loss: 2.2737
Epoch 2/15
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.4520 - loss: 1.5620
Epoch 2: val_loss did not improve from 2.27375
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 0.4548 - loss: 1.5552 - val_accuracy: 0.1020 - val_loss: 2.5654
Epoch 3/15
31/32 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.8695 - loss: 0.5427
Epoch 3: val_loss did not improve from 2.27375
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 0.8716 - loss: 0.5374 - val_accuracy: 0.1020 - val_loss: 2.5708
Epoch 4/15
30/32 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.9755 - loss: 0.2100
Epoch 4: val_loss did not improve from 2.27375
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 0.9763 - loss: 0.2073 - val_

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Get predictions (class indices for each sample)
pred = bigru_cnn_model.predict(test_data)
y_pred = np.argmax(pred, axis=1)  # Get the class with the highest probability

# Calculate metrics
precision = precision_score(y_test, y_pred, average='micro')  # Micro-average precision
recall = recall_score(y_test, y_pred, average='micro')        # Micro-average recall
f1 = f1_score(y_test, y_pred, average='micro')                # Micro-average F1 score
accuracy = accuracy_score(y_test, y_pred)                     # Accuracy

# Print results
print("Precision (Micro):", precision)
print("Recall (Micro):", recall)
print("F1 Score (Micro):", f1)
print("Accuracy:", accuracy)


7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 63ms/step
Precision (Micro): 0.035
Recall (Micro): 0.035
F1 Score (Micro): 0.035
Accuracy: 0.035
